# New York City Taxi Ride Duration Prediction

In this case study, we will build a predictive model to predict the duration of taxi ride. We will do the following steps:
  * First install the dependencies
  * Next load the data as pandas dataframe
  * Define the outcome variable - the variable we are trying to predict.
  * Build features using [featuretools](featuretools.com) package that implements Deep Feature Synthesis. We will start with simple features and incrementally improve the feature definitions and examine the accuracy of the system.


Allocate at least 2-3 hours to go through this case study end-to-end

# Install Dependencies 
<p>If you have not done so already, download this repository <a href="https://github.com/Featuretools/DSx/archive/master.zip">from git</a>. Once you have downloaded this archive, unzip it and cd into the directory from the command line. Next run the command ``./install_osx.sh`` if you are on a mac or ``./install_linux.sh`` if you are on linux. This should install all of the dependencies.</p>
<p> If you are on a windows machine, open the requirements.txt folder and make sure to install each of the dependencies listed (featuretools, jupyter, pandas, sklearn, numpy) </p>
<p> Once you have installed all of the dependencies, open this notebook. On Mac and Linux, navigate to the directory that you downloaded from git and run ``jupyter notebook`` to be taken to this notebook in your default web browser. When you open the NewYorkCity_taxi_case_study.ipynb file in the web browser, you can step through the code by clicking the ``Run`` button at the top of the page. If you have any questions for how to use <a href="http://jupyter.org/">Jupyter</a>, refer to google or the discussion forum.</p>

# Running the Code

In [1]:
import featuretools as ft
import utils
from utils import load_nyc_taxi_data, compute_features, preview, feature_importances
from sklearn.ensemble import GradientBoostingRegressor
from featuretools.primitives import (Day, Hour, Minute, Month, Weekday, 
                                     Week, Weekend, Sum, Mean, Median, Std)
print ft.__version__
%load_ext autoreload
%autoreload 2

0.1.14
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Step 1: Download and load the raw data as pandas dataframes
<p>If you have not yet downloaded the data it can be downloaded <a href="https://s3.amazonaws.com/mit-dsx-data/nyc-taxi-data.zip">from S3</a>. Once you have downloaded the archive, unzip it and place the nyc-taxi-data folder in the same directory as this script. 
</p>

In [2]:
trips, passenger_cnt, vendors = load_nyc_taxi_data()
preview(trips,10)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,trip_duration
0,0,2,2016-01-01 00:00:19,2016-01-01 00:06:31,3,1.32,-73.961258,40.796200,False,-73.950050,40.787312,2,372.0
679995,679995,1,2016-04-30 12:57:36,2016-04-30 13:04:36,1,1.10,-73.979973,40.770679,False,-73.969696,40.785587,1,420.0
679996,679996,2,2016-04-30 12:57:40,2016-04-30 13:06:01,5,1.22,-73.940399,40.793880,False,-73.952667,40.804859,1,501.0
679997,679997,2,2016-04-30 12:57:45,2016-04-30 13:07:11,1,1.58,-73.924728,40.744068,False,-73.953087,40.749290,1,566.0
679998,679998,2,2016-04-30 12:57:49,2016-04-30 13:15:28,1,4.76,-73.985863,40.746799,False,-74.005951,40.711269,2,1059.0
679999,679999,2,2016-04-30 12:58:04,2016-04-30 13:08:30,1,1.79,-73.959747,40.773682,False,-73.981071,40.778381,2,626.0
680000,680000,2,2016-04-30 12:58:33,2016-04-30 13:08:51,1,1.32,-73.981300,40.752972,False,-73.973923,40.764381,1,618.0
680001,680001,2,2016-04-30 12:58:39,2016-04-30 13:16:19,2,1.99,-73.987549,40.756226,False,-73.998032,40.765732,2,1060.0
680002,680002,2,2016-04-30 12:58:47,2016-04-30 13:13:47,2,4.00,-73.951172,40.774220,False,-73.909988,40.801823,1,900.0
680003,680003,1,2016-04-30 12:58:56,2016-04-30 13:24:28,1,6.10,-74.008163,40.703640,False,-73.984138,40.758980,2,1532.0


The ``trips`` table has the following fields
* ``id`` which uniquely identifies the trip
* ``vendor_id`` is the taxi cab company - in our case study we have data from three different cab companies
* ``pickup_datetime`` the time stamp for pickup
* ``dropoff_datetime`` the time stamp for drop-off
* ``passenger_count`` the number of passengers for the trip
* ``trip_distance`` total distance of the trip in miles 
* ``pickup_longitude`` the longitude for pickup
* ``pickup_latitude`` the latitude for pickup
* ``dropoff_longitude``the longitude of dropoff 
* ``dropoff_latitude`` the latitude of dropoff
* ``payment_type`` A numeric code signifying how the passenger paid for the trip. 1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided
* ``trip_duration`` this is the duration we would like to predict using other fields 

# Step 2: Prepare the Data
Lets create entities and relationships. The three entities in this data are 
* trips 
* vendors (these are the cab companies)
* passenger_cnt (a simple entity that has the unique number of passenger counts 1-8)

This data has the following relationships
* Vendors --> trips (the same vendor can have multiple trips - vendors is the ``parent_entity`` and trips it the child entity
* passenger_cnt --> trips (the same passenger_cnt can appear in multiple trips. passenger_cnt is the ``parent_entity`` and trips is the child entity. 

In <a <href="https://www.featuretools.com/"><featuretools (automated feature engineering software package)/></a>, we specify the list of entities and relationships as follows: 

In [3]:
entities = {
        "trips": (trips, "id", 'pickup_datetime' ),
        "vendors": (vendors, "vendor_id"),
        "passenger_cnt": (passenger_cnt,"passenger_count")
        }

relationships = [("vendors", "vendor_id","trips", "vendor_id"), 
                ("passenger_cnt", "passenger_count","trips", "passenger_count")]

<p>We specify the time for each instance of the target_entity, in this case ``trips`` to calculate features. The timestamp represents the last time data can be used for calculating features by DFS. This is specified using a dataframe of cutoff time. This cutoff time for each trip is the pickup time. We want to have a minimum amount of data, so we only use trips after January 12th, 2016</p>

In [4]:
cutoff_time = trips[['id', 'pickup_datetime']]
cutoff_time = cutoff_time[cutoff_time['pickup_datetime'] > "2016-01-12"]
preview(cutoff_time, 10)

,id,pickup_datetime
56311,56311,2016-01-12 00:00:25
698765,698765,2016-05-03 18:54:53
698766,698766,2016-05-03 18:55:37
698767,698767,2016-05-03 18:55:38
698768,698768,2016-05-03 18:55:49
698769,698769,2016-05-03 18:55:58
698770,698770,2016-05-03 18:56:22
698771,698771,2016-05-03 18:56:24
698772,698772,2016-05-03 18:56:51
698773,698773,2016-05-03 18:56:56


# Step 3: Create baseline features using DFS 
<p>Instead of manually creating features, such as "<b>month of pickup_datetime</b>", we can let featuretools come up with them. </p> 

Featuretools does this by 
* interpret the types of variables - categorical, numeric and others. We can override this interpretation by specifying the types. In this case study, we wanted <b>passenger_count</b> to be a type of Ordinal, and <b>vendor_id</b> to be of type Categorical. This override occured while loading in the csv files.</p>
* then based on the primitives we specify, it matches up the columns to which those primitives can be applied. 

# Create transform features using transform primitives

As we described in the video, features fall into two major categories, ``transform`` and ``aggregate``. In featureools, we can create transform features by specifying ``transform`` primitives. Below we specify a ``transform`` primitive called ``weekend`` and here is what it does:

* It can be applied to any ``datetime`` column in the data. 
* For each entry in the column, it assess if it is a ``weekend`` and returns a boolean. 

In this specific data, there are two ``datetime`` columns ``pickup_datetime`` and ``dropoff_datetime``. The tool automatically creates features using the primitive and these two columns as shown below. 

In [5]:
trans_primitives = [Weekend]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=[],
                  features_only=True)

<p>Here are the features created.</p>

In [6]:
print len(features)
features

12


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: payment_type>,
 <Feature: dropoff_longitude>,
 <Feature: pickup_latitude>,
 <Feature: trip_duration>,
 <Feature: store_and_fwd_flag>,
 <Feature: trip_distance>,
 <Feature: dropoff_latitude>,
 <Feature: pickup_longitude>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>]

Now let's compute the features. 

In [7]:
feature_matrix = compute_features(features, cutoff_time)

Progress: 100%|██████████| 5/5 [00:07<00:00,  1.42s/cutoff time]


# Step 4: Build the Model

To build a model,
* we first seperate the data into a porition for ``training`` (75% in this case) and a portion for ``testing`` 
* We also get the log of the trip duration so that a more linear relationship can be found.
* We use ``GradientBoostingRegressor`` to train a model. 

In [8]:
# separates the whole feature matrix into train data feature matrix, 
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)

In [9]:
model = GradientBoostingRegressor()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.68784144118572588

# Step 5: Adding more Transform Primitives

* Add ``Minute``, ``Hour``, ``Week``, ``Month``, ``Weekday`` primitives
* All these transform primitives apply to ``datetime`` column

In [10]:
trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, Weekend]

features = ft.dfs(entities=entities,
                   relationships=relationships,
                   target_entity="trips",
                   trans_primitives=trans_primitives,
                   agg_primitives=[],
                   features_only=True)

In [11]:
print len(features)
features

36


[<Feature: passenger_count>,
 <Feature: dropoff_longitude>,
 <Feature: payment_type>,
 <Feature: store_and_fwd_flag>,
 <Feature: vendor_id>,
 <Feature: pickup_latitude>,
 <Feature: pickup_longitude>,
 <Feature: trip_duration>,
 <Feature: trip_distance>,
 <Feature: dropoff_latitude>,
 <Feature: WEEKDAY(pickup_datetime)>,
 <Feature: WEEK(dropoff_datetime)>,
 <Feature: HOUR(pickup_datetime)>,
 <Feature: WEEKDAY(dropoff_datetime)>,
 <Feature: DAY(pickup_datetime)>,
 <Feature: MONTH(pickup_datetime)>,
 <Feature: WEEK(pickup_datetime)>,
 <Feature: DAY(dropoff_datetime)>,
 <Feature: MONTH(dropoff_datetime)>,
 <Feature: HOUR(dropoff_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: MINUTE(pickup_datetime)>,
 <Feature: MINUTE(dropoff_datetime)>,
 <Feature: passenger_cnt.WEEK(first_trips_time)>,
 <Feature: vendors.DAY(first_trips_time)>,
 <Feature: passenger_cnt.WEEKDAY(first_trips_time)>,
 <Feature: vendors.WEEKDAY(first_trips_time)>,
 <Fe

Now let's compute the features. 

In [12]:
feature_matrix = compute_features(features, cutoff_time)

Progress: 100%|██████████| 5/5 [00:21<00:00,  4.35s/cutoff time]


In [13]:
preview(feature_matrix, 10)

,passenger_count,dropoff_longitude,payment_type,store_and_fwd_flag,vendor_id,pickup_latitude,pickup_longitude,trip_duration,trip_distance,dropoff_latitude,...,passenger_cnt.WEEKDAY(first_trips_time),vendors.WEEKDAY(first_trips_time),vendors.MONTH(first_trips_time),passenger_cnt.DAY(first_trips_time),passenger_cnt.MINUTE(first_trips_time),passenger_cnt.HOUR(first_trips_time),vendors.HOUR(first_trips_time),passenger_cnt.MONTH(first_trips_time),vendors.MINUTE(first_trips_time),vendors.WEEK(first_trips_time)
id,,,,,,,,,,,,,,,,,,,,,
56311,1,-74.004730,1,False,2,40.719952,-73.987396,645.0,1.61,40.722809,...,4,4,1,1,45,1,0,1,1,53
698765,1,-73.984131,1,False,1,40.750095,-73.977142,227.0,0.70,40.745785,...,4,4,1,1,45,1,0,1,1,53
698766,1,-73.993301,1,False,1,40.721073,-73.997627,477.0,1.00,40.733433,...,4,4,1,1,45,1,0,1,1,53
698767,1,-73.945930,1,False,2,40.768250,-73.982140,1371.0,5.13,40.724018,...,4,4,1,1,45,1,0,1,1,53
698768,1,-73.952881,1,False,2,40.756191,-73.964592,350.0,1.29,40.772099,...,4,4,1,1,45,1,0,1,1,53
698769,1,-73.978600,1,False,1,40.741943,-73.985123,312.0,0.70,40.751209,...,4,4,1,1,45,1,0,1,1,53
698770,1,-73.975525,1,False,2,40.763371,-73.985252,287.0,0.72,40.765606,...,4,4,1,1,45,1,0,1,1,53
698771,1,-73.982208,2,False,1,40.741833,-73.992401,454.0,0.60,40.737656,...,4,4,1,1,45,1,0,1,1,53
698772,1,-74.010147,1,False,2,40.774170,-73.872910,1715.0,12.43,40.709469,...,4,4,1,1,45,1,0,1,1,53


# Step 6: Build the new model

In [ ]:
# separates the whole feature matrix into train data feature matrix,
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)

In [ ]:
model = GradientBoostingRegressor()
model.fit(X_train,y_train)
model.score(X_test,y_test)

# Step 7: Add Aggregation Primitives

Now let's add aggregation primitives. These primitives will generate features for the parent entities in this case both ``vendors`` and ``passenger_cnt`` and then add them to the trips entity (which is the entity for which we are trying to make prediction.

In [ ]:
trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, Weekend]
aggregation_primitives = [Sum, Mean, Median, Std]

features = ft.dfs(entities=entities,
                   relationships=relationships,
                   target_entity="trips",
                   trans_primitives=trans_primitives,
                   agg_primitives=aggregation_primitives,
                   features_only=True)

In [ ]:
print len(features)
features

In [ ]:
feature_matrix = compute_features(features, cutoff_time)

In [ ]:
preview(feature_matrix, 10)

# Step 8: Build the new model

In [ ]:
# separates the whole feature matrix into train data feature matrix,
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)

In [ ]:
model = GradientBoostingRegressor()
model.fit(X_train,y_train)

# Step 9: Evalute on test data

In [ ]:
y_pred = model.predict(X_test)
y_pred[5:]

In [ ]:
model.score(X_test,y_test)

# Additional Analysis
<p>Let's look at how important each feature was for the model.</p>

In [ ]:
feature_importances(model, feature_matrix.columns.tolist(), n=15)